In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

# Python私房手册-函数,类元类,模块包和异常

## 函数

### 函数中的try...finally中的return

今天发现一个很有意思的代码，就是当函数中return在try...finally的try中，finally的代码仍然会执行：

In [13]:
def func():
    try:
        return 42
    finally:
        print("in finally")
        
func()

in finally


42

在14.1.3中有个利用finally在queue上调用task_done的例子。

### 函数默认参数

函数默认参数很容易犯的一个错误是，默认等于一些可变的对象，即空列表，或者空字典之类的。这样的话会导致一些奇怪的结果，如：

In [13]:
def func(a, b=[]):
    b.append(a)
    return b

l1 = func(2)
l1.append("hello world")
l2 = func(2)
print(l2)

[2, 'hello world', 2]


惊不惊喜，意不意外？因为函数的默认参数是在定义时进行赋值的，且只赋值一次，即在编译的时候，`func`就创造了一个内部变量`b=[]`，如果在函数执行的过程当中，没有修改过这个变量`b`，那么每次执行函数，这个变量`b`始终引用的是同一个可变对象。

### 匿名函数捕获变量

和默认参数是在定义时绑定相对的，是匿名参数的内部变量是在函数运行时绑定，比如：

In [100]:
funcs = [lambda x: x+n for n in range(3)]
for f in funcs:
    f(1)

3

3

3

因为函数内部的`n`并不是外面的变量`n`，在列表内循环的时候并没有什么赋值行为的发生。当在`for f in funcs`循环中，执行`f(1)`的时候，当执行到`x+n`时，才会查找变量`n`的值，此时在函数内部中未找到，转而到外层去找，此时可以找到，但是此时外层的`n`是2，所以输出全部都是3。

要解决很简单，就是利用函数默认参数在定义时绑定的特性，设置一个默认参数：

In [102]:
funcs = [lambda x, n=n:x+n for n in range(3)]
for f in funcs:
    f(1)

1

2

3

### 什么时候用`nonlocal`

今天写一段代码发生了错误，简化的代码如下：
```python
def func():
    nonlocal var
    print(var)
    
def printvar():
    var = 42
    func()
    
printvar()
```
结果出现了语法错误：`SyntaxError: no binding for nonlocal 'var' found`，提示没有发现`nonlocal var`可以绑定的变量。所以，`nonlocal`只能在嵌套函数中使用，表示该变量属于外层函数，如下：
```python
def func():
    def printvar():
        nonlocal var
        print(var)
    
    var = 42
    printvar() 
    
func()  # 输出42
```
有意思的是，上面的程序如果是脚本，可以正常运行，但如果是命令行模式，同样会抛出`SyntaxError: no binding for nonlocal 'var' found`错误，推测是因为脚本和命令行模式编译的顺序不一样，对于脚本而言，会将函数作为一个整体由外向内的进行编译，即先声明外层函数的所有变量，再逐层声明内部函数体内的变量，从而可以顺利将`var`与外层函数的变量进行绑定，而命令行是逐行编译，当运行到`nonlocal var`语句时，外层函数还没有声明`var`变量，因此会报错。所以，在命令行中，`var = 42`语句需要放在`def printvar()`函数定义的前面。

这里其实并不需要`nonlocal var`语句，如下：
```python
def func():
    def printvar():
        print(var)
    
    var = 42
    printvar()

func()  # 输出42
```
此时，脚本模式和命令行模式都可以顺利运行，因为没有`nonlocal`关键字，命令行模式并不会去检查`printvar`内部的`var`变量与谁绑定，因此不会报错。
在这里，不声明`nonlocal`似乎更好，那么，什么情况下需要使用`nonlocal`声明呢？答案是，当内层函数需要修改外层函数的变量时，必须要使用`nonlocal`声明。如下：

In [4]:
def sort_priority(numbers, group):
    found = False
    def helper(x):
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [2, 5, 1, 8, 10]
sort_priority(numbers, [6, 7, 8, 9, 10])
print(numbers)

False

[8, 10, 1, 2, 5]


上面的代码是给出一个组，如果`numbers`内的元素在`group`中，则优先排序。返回的`numbers`排序是对的，但是`found`值不对，因为这里返回的其实是外层函数的`found`，内层函数通过赋值，创建了一个属于内层函数的`found`变量。而内层函数的本意是修改外层函数的`found`变量，因此这里必须要使用`nonlocal`声明，如下：

In [5]:
def sort_priority(numbers, group):
    found = False
    def helper(x):
        nonlocal found
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [2, 5, 1, 8, 10]
sort_priority(numbers, [6, 7, 8, 9, 10])
print(numbers)

True

[8, 10, 1, 2, 5]


最后总结一下：
1. `nonlocal`只能在嵌套函数的内层函数中使用，表明该变量属于外层函数。
2. `nonlocal`主要用来在内层函数中修改外层函数的变量的值，如果仅仅只是引用外层函数的变量而不修改，根据作用域查找规则，可以不使用`nonlocal`。

### 如何修改函数签名

修改函数签名可以使用inspect模块，如下：

In [2]:
import inspect

def add(a, b):
    return a + b

sig = inspect.signature(add)
params = list(sig.parameters.values())
# 在sig.parameters.values()里添加或者删除即可
params.append(inspect.Parameter('c', inspect.Parameter.POSITIONAL_OR_KEYWORD))
# sig.replace返回一个新的签名，原来的签名不变
new_sig = sig.replace(parameters=params)
# sig.parameters是一个mappingproxy，修改了parameters.values()，则parameters也发生了变化
print(new_sig.parameters)
add.__signature__ = new_sig

OrderedDict([('a', <Parameter "a">), ('b', <Parameter "b">), ('c', <Parameter "c">)])


注释标注的几点要注意，另外虽然修改了函数签名，但是函数实际接收的参数并未发生变化，修改函数签名主要用在当装饰器添加了额外的关键字参数时，需要修改函数签名以保证装饰器和包装函数签名保持一致。具体例子可见《python cookbook》9.11一节。

另外，inspect还有一个getfullargspec函数，可以快速获取函数的所有参数，用来判断是否包含某个参数比较方便，如下：

In [77]:
argspec = inspect.getfullargspec(add)
print(argspec)
'c' in argspec.args

FullArgSpec(args=['a', 'b', 'c'], varargs=None, varkw=None, defaults=None, kwonlyargs=[], kwonlydefaults=None, annotations={})


True

也可以用sig.parameters.keys()来判断：

In [78]:
'c' in new_sig.parameters.keys()

True

### 函数的`__code__.varnames`,`__code__.cellvars`和`__code__.freevars`属性

- `varnames`包含函数的本地变量名。
- `cellvars`包含函数被内部函数引用了的所有变量，外层函数的`cellvars`和内层函数的`freevars`是一致的。对于外层函数来说，varnames+cellvars才是这个函数本地作用域的所有变量。
- `freevars`内层函数引用的外层函数的变量名。

In [8]:
def outer():
    a = 42
    def inner():
        b = 24
        print(a)
    return inner

inner = outer()
outer.__code__.co_varnames
outer.__code__.co_cellvars
outer.__code__.co_freevars
inner.__code__.co_varnames
inner.__code__.co_cellvars
inner.__code__.co_freevars

('inner',)

('a',)

()

('b',)

()

('a',)

In [4]:
c = inner.__closure__[0]

In [9]:
c.cell_contents

42

In [7]:
import inspect

In [8]:
inspect.getclosurevars(inner)

ClosureVars(nonlocals={'a': 42}, globals={}, builtins={'print': <built-in function print>}, unbound=set())

## 常用内置函数

### `round()`

第2个参数表示精度，如下的例子，-1表示四舍五入到10位：

In [110]:
round(11, -1)
round(16, -1)

10

20

### `eval() exec() global() locals()`以及`compile()`

- [Python中的eval()、exec()及其相关函数](https://www.cnblogs.com/yyds/p/6276746.html)
- [What's the difference between eval, exec, and compile?](https://stackoverflow.com/questions/2220699/whats-the-difference-between-eval-exec-and-compile/29456463#29456463)

`eval()`和`exec()`可以将字符串当作代码来执行，相当于可以动态的产生并执行代码，`eval`一般执行简单的表达式，并且有返回值，`exec`可以执行复杂的语句，但是返回值永远是`None`，第二个参数、第三个参数分别用来指定全局作用域和局部作用域，它们的最大区别实际上来自于`compile`及其模式:
1. 可以先将字符串`compile()`成字节码，然后再`eval()`或者`exec()`,这可以加速对相同代码的反复调用。`compile`第三个参数接收`exec`,`eval`和`single`三种模式，返回一个`code`对象。实际上，如果将包含源代码的`str/unicode/bytes`传递给`eval`或者`exec`，它的行为相当于`eval(compile(source, '<string>', 'eval'))`或者`exec(compile(source, '<string>', 'exec'))`。
2. 可以先`compile(source, '<string>', 'exec')`,将返回的`code`对象传递给`eval`，这样使用`eval`也可以执行语句，不过返回的值为`None`，此时`eval`和`exec`没有区别。

## 类和对象

### 容易出错的`__getattribute__`

有以下几点要注意：
1. 只有通过`instance.attr`这种形式访问属性的时候才会触发`__getattribute__`，在类中查找则不会触发。所以如果是`C.attr(c)`这样的调用，不会触发。
2. 大多数特殊方法直接在类中查找属性，不会触发，因此编写委托类的时候，这些特殊方法需要重写。但是`dir`比较特殊，从实例中查找，因此会触发。
3. `hasattr`函数也会触发`__getattribute__`。

2022年1月4日补充：  
一直没有搞懂`object.__getattribute__`本身是如何实现的，为什么它的内部不会触发递归。不过从下面代码可以稍微推测一二，访问任何实例属性都会先调用`object.__getattribute__`，`object.__getattribute__`会从实例的命名空间开始查找，即它会访问实例的`__dict__`，而不会触发调用自身导致递归。如果是普通的属性，则返回值。如果是函数，则会进行包装，返回一个绑定的方法，本质上是一个描述符。

In [6]:
o = object()

In [7]:
# 可以看到，本质上是一个method-wrapper，方法包装器
o.__getattribute__

<method-wrapper '__getattribute__' of object object at 0x000001A31E75BA70>

In [32]:
class C:
    def __len__(self):
        return 42
    
    def __getattribute__(self, attr):
        print('invoke getattribute')

In [19]:
c = C()

invoke getattribute
invoke getattribute
invoke getattribute
invoke getattribute


In [20]:
# 访问任何的普通属性都会先调用__getattribute__
c.name

invoke getattribute


In [21]:
# 只要是通过`实例.属性`这种方式访问属性就会触发，`.`本质上是个运算符，相当于调用实例的`__getattribute__`方法
c.__le__

invoke getattribute


In [22]:
# 这样访问实际上会直接在类的命名空间查找方法，因此不会触发
len(c)

42

In [25]:
# 直接通过类访问也不会触发
C.__len__(c)

42

In [26]:
class C:
    def name():
        pass
    def __getattribute__(self, attr_name):
        print("invoke")
        # 或者 return object.__getattribute__(self, attr_name)
        return super().__getattribute__(attr_name)

In [28]:
c = C()
# 由__getattribute__完成对函数的包装，返回一个绑定了实例的方法
c.name

invoke


<bound method C.name of <__main__.C object at 0x000001A31E8A50D0>>

### 关于`super()`

`super()`一直是觉得比较神奇的一个方法，特别是不带参数的时候，会神奇的自动绑定相应的对象。但是太神奇了，有时候容易出错，而且错误不容易排查。因此做一个梳理。`super()`返回一个带`__get__`方法的描述符对象，主要有2点要注意：
1. `super()`在多继承的时候按照`mro`的顺序调用父类。

In [90]:
class Base:
    def __init__(self):
        print("Base inited.")


class ChildA(Base):
    def __init__(self):
        super().__init__()
        print("ChildA inited")


class ChildB(Base):
    def __init__(self):
        super().__init__()
        print("ChildB inited")


class User(ChildA, ChildB):
    def __init__(self):
        super().__init__()


User.mro()
u = User()

[__main__.User, __main__.ChildA, __main__.ChildB, __main__.Base, object]

Base inited.
ChildB inited
ChildA inited


In [83]:
class Base:
    def __init__(self):
        print("Base inited.")


class ChildA(Base):
    def __init__(self):
        Base.__init__(self)
        print("ChildA inited")


class ChildB(Base):
    def __init__(self):
        Base.__init__(self)
        print("ChildB inited")


class User(ChildA, ChildB):
    def __init__(self):
        super().__init__()


# User的super()返回的是ChildA，ChildA中直接调用Base，因此不会调用ChildB中的__init__方法
u = User()

Base inited.
ChildA inited


2. `super()`总是尽可能的将正确的参数传递给方法，在内部做了很多工作，但是这样有时候反而容易让人疑惑。

In [88]:
class C:
    data = "data"

    def method(*args):
        print(args)

    @classmethod
    def cmethod(*args):
        print(args)

    @staticmethod
    def smethod(*args):
        print(args)


class SubC(C):
    def __init__(self):
        super().method()
        super().cmethod()
        super().smethod()
        print("=" * 50)
        super(SubC, self).method()  # 正确，将self传递给方法
        super(SubC, self).cmethod()  # 正确，虽然调用的是super(SubC, self)，但是传入方法的不是self而是class
        super(SubC, self).smethod()  # 正确，没有参数传递
        print("=" * 50)
        super(SubC, SubC).method()  # 错误，没有参数传递给方法
        super(SubC, SubC).cmethod()  # 正确，将class传递给方法
        super(SubC, SubC).smethod()  # 正确，没有参数传递


sc = SubC()

(<__main__.SubC object at 0x000002516734B240>,)
(<class '__main__.SubC'>,)
()
(<__main__.SubC object at 0x000002516734B240>,)
(<class '__main__.SubC'>,)
()
()
(<class '__main__.SubC'>,)
()


由此可见，`super`并不是简单的将第二个参数传递给方法，而是在内部进行了判断，尽可能的将正确的对象传递给方法，唯一的例外是用`super(type, type2)`方式调用实例方法的时候，由于`super()`不知道要传递给方法的实例是什么，此时会什么都不传。其它情况下，都能传递正确的对象给方法。  

仔细思考如下代码，加深理解：

In [119]:
from inspect import signature


class Base:
    def func(self):
        print(self.x + self.y)


class C(Base):
    def func(self):
        self.x = 2
        self.y = 3
        super(C, self).func()


c = C()
c.func()

5


#### 子类扩展property

《python cookbook》子类扩展property中，在子类中获取父类的特性使用`super().name`，而修改使用的是`super(C, C).name.__set__(self, value)`，始终没太明白修改为什么不能直接`super().name = value`，测试如下： 

In [71]:
class C:
    cname = "classname"
    
    def __init__(self, name):
        self._name = name
        
    @property
    def name(self):
        return self._name
    
    @name.setter
    def name(self, value):
        self._name = value

class SubC(C):
    def func(self, value):
        print(super()) # <super: <class 'SubC'>, <SubC object>>
        print(hasattr(super(), '__get__'))  # True
        print(hasattr(super(), '__set__'))  # False
        print(super(SubC, SubC)) # <super: <class 'SubC'>, <SubC object>>
        print(super(SubC, SubC).name)  # <property object at 0x000001D59FEADC78>
        print(super(SubC, SubC).cname)  # classname
        print(super().name)  # shy
        print(super().cname)  # classname
        # super().cname = value  # 抛出AttributeError: 'super' object has no attribute 'cname'错误
        # super().name = value   # 同样抛出AttributeError: 'super' object has no attribute 'name'错误

super的内部机制还是没有完全弄明白，根据代码猜测，super()会返回一个super类的实例，对这个super实例的属性进行查找，相当于在父类（例子中的C）中查找属性，猜测在内部super复制了SubC的MRO列表？并且如果属性是方法的话，会传递正确的参数（self或者cls）给方法。查找仅仅针对读操作，任何写操作会直接给super实例进行赋值，不会在MRO中查找属性，因此会出现`'super' object has no attribute 'cname'`的错误，想想也对，不可能在子类中修改父类的属性或者方法。

再看问题，如果继承的属性是父类的property或者描述符，子类中super().name相当于调用`C.name.__get__(self, SubC)`，注意，self是子类的实例，而super(SubC, SubC)相当于`C.name.__get__(None, SubC)`，父类的name是一个描述符，调用它实际上相当于调用它的__get__方法。因此super().name传递了子类的实例，返回字符串，而`super(SubC, SubC)`由于根本没有不知道子类的实例是啥，因此向__get__方法传递是None，因此返回的是描述符本身。

2022年1月4日补充：  
可以这样理解，`super()`本身是一个实例S，它相当于某个父类C的代理，所有对S的属性的获取，它会全部全部代理给C。但是它没有代理属性的赋值，所有属性赋值操作相当于直接对S的属性进行赋值。所以会抛出错误。

### `__new__`方法以及在`__new__`方法中使用`super()`的一个bug

`__new__`方法有3点需要注意：
1. `__new__`方法接收的第一个参数是类本身，`__new__`是一个静态方法而不是类方法，所以不存在绑定一说，不会自动传入第一个参数。
2. 当`super()`方法省略参数的时候，`super()`方法会在堆栈中寻找类(`__class__`)以及第一个参数。
>The zero argument form automatically searches the stack frame for the class (__class__) and the first argument.
3. 如果在`__new__`中使用`super()`并且省略参数，必须显示的传入第一个`cls`参数，不能直接使用动态参数来传递`cls`(比如`*args`)，否则会报错（可以视为一个bug），或者显式的使用`super(type, obj_or_type)`方式进行调用。

In [275]:
class C:
    def __new__(*args):
        print("Create c object.")
        # 注意不能 return super().__new__(*args)，这样会报错
        # 注意，这种形式args不能包含其它参数，因为super(C, C)其实调用的是object.__new__(cls)，只接收一个cls参数。
        return super(C, C).__new__(*args)
    
#     # 也可以是下面这种形式
#     def __new__(cls, *args):
#         print("Create c object")
#         # 如果继承了其它的类，则要根据父类的情况传递参数
#         return super().__new__(cls)

c = C()

Create c object


4. 一般情况下，普通的类中，`__new__`中的`super()`指的是`object`，而元类中的`super()`指的是`type`，两者接收的参数是不一样的。

In [20]:
class C:
    def __new__(cls, *args, **kwargs):
        print(args)
        print(kwargs)
        # 注意，super()指的是object，创建的是实例，只接收一个cls的参数
        return super().__new__(cls)


c = C(1, 2, a=3)

(1, 2)
{'a': 3}


In [47]:
sc = SubC("shl")

In [22]:
class MC(type):
    def __new__(mcls, *args, **kwargs):
        print(args)
        print(kwargs)
        # 此时super()指的是type，创建的是类，接收的参数是固定，分别是元类，类名称，父类元组，属性字典
        return super().__new__(mcls, *args, **kwargs)


class C(metaclass=MC):
    pass


c = C()

('C', (), {'__module__': '__main__', '__qualname__': 'C'})
{}


### `__new__`不返回实例的话？？

今天考试有一道题是，`__new__`如果不返回实例，则`__init__`不会调用，觉得很有意思，以前没有思考过。可以测试一下：

In [7]:
class C:
    def __new__(cls, *args, **kwargs):
        print("in new:", args, kwargs)
    
    def __init__(self, *args, **kwargs):
        print("in init:", args, kwargs)
        print(self)  

In [8]:
c = C()

in new: () {}


可以看到，如果`__new__`不返回实例或者说返回任何不是实例的东西，则`__init__`根本不会被调用。

### `staticmethod`静态方法

要注意的是`staticmethod`静态方法实际就相当于普通的函数，《流畅的python》对`staticmethod`的看法是：可有可无，当感觉需要定义一个静态方法的时候，完全可以把它放在模块级别而不放在类的内部。  
这里有几点需要注意的是：
1. `staticmethod`可以用实例，也可以用类进行调用。
2. 在类的外部和类的内部，调用方式是一样的。

In [3]:
class C:
    @staticmethod
    def sfunc():
        print("I am staticmethod.")
    
    def ifunc(self):
        print("in func:")
        self.sfunc()
        C.sfunc()
        print("end")

c = C()
c.ifunc()

in func:
I am staticmethod.
I am staticmethod.
end


### 是函数还是方法

理解函数和方法的区别需要理解`python`的描述符协议，简单来说，通过实例访问依附在类上的函数时， 经由描述符协议的处理， 就会变成方法，方法其实是一个名为`method`的描述符类。注意2个要点：1、依附在类上。2、遵循描述符协议，即拥有`__get__`方法，如下的代码，在类的外部定义方法：

In [51]:
def func(self, data):
    self.data = data
    return self.data

class C:
    def __init__(self):
        pass

C.func = func
c1 = C()
c1.func(42)

42

方法只能依附于类，不能依附于实例，如下：

In [52]:
c1.func = func
try:
    c1.func(42)
except TypeError as e:
    print(e)
    
c1.func

func() missing 1 required positional argument: 'data'


<function __main__.func(self, data)>

可见，`self`参数并未正确的传递，此时`c1.func`仍然是一个普通的函数，而不是绑定的方法，只有当把`func`作为类`C`的属性，通过实例`c`访问的时候，`func`才是方法。注意，此时对类`C`来说，`func`是未绑定的，仍然是一个函数（严格来说，此时也是调用了函数的`__get__`方法，不过由于传入的实例是`None`，返回函数自身）：

In [53]:
del(c1.func)
c1.func
C.func

<bound method func of <__main__.C object at 0x0000025E347F85E0>>

<function __main__.func(self, data)>

我们来看看在底层，`python`解释器到底做了什么，通过实例访问的时候，先触发类的`__getattribute__`方法，直接返回一个绑定了实例的`method`对象，等同于像下面这样手动创建的`method`对象：

2020年12月21日补，访问`__getattribute__`这样的特殊方法时，不会从实例查找，而是直接以`inst.__class__.__getattribute__(inst)`的形式进行调用。

In [54]:
bm = C.func.__get__(c1)
bm
bm(42)

<bound method func of <__main__.C object at 0x0000025E347F85E0>>

42

<font color="red">这里有一点没明白，即使是覆盖了`func`的`__get__`方法，通过实例访问，仍然能返回正确的`method`对象，可见内部的机制不仅仅是简单的调用`C.func.__get__(c1)`</font>  
回答：通过实例访问，在内部并不是像调用普通的描述符那样，调用`C.func.__get__`，而是调用了类的`__getattribute__`方法，返回一个`method`对象。所以覆盖了`func`的`__get__`方法，仍然会返回正确的`method`对象，`method`对象也有`__get__`方法，而且这个方法不能修改是只读的。

这里的理解有误，func是function类的实例，当访问`__get__`这种特殊方法的时候，会直接在类的命名空间内查找，而不会查找实例的命名空间。因此覆盖了func的`__get__`方法没有用，当通过`C.func.__get__(c1)`调用时，调用的实际上是function类的`__get__`方法，而不是func这个function实例的`__get__`方法。

`method`对象有几个特殊的属性，分别是`__call__`，`__self__`，`__func__`，调用这个对象的时候，实际上执行的是这个对象的`__call__`方法，另外，`__self__`包含的是对象依附的类，`__func__`包含的是原始的函数:

In [55]:
bm.__self__
bm.__func__
bm.__call__(42)

<function __main__.func(self, data)>

42

在`__call__`方法内部，会将之前绑定的`c`实例作为第一个参数传递给`func`函数（也就包含在对象的`__func__`属性中的原始函数），并执行。所以为什么说是已绑定的方法，是因为这个`method`对象和实例`c`已经绑定了。

以上是`python`解释器的内部运行机制，因为函数是依附在类上的，不同的实例访问同一个函数的时候，都会自动将这个实例与函数进行绑定，如果只想绑定某一个特定的实例的话，一种方法是像上面这样，通过函数的`__get__`方法指定要绑定的实例：

In [56]:
c2 = C()
bm2 = func.__get__(c2)
bm2("foo")

'foo'

还有一种方法是使用`python`的`types`模块的`MethodType`方法来手动绑定：

In [70]:
from types import MethodType

c3 = C()
bm3 = MethodType(func, c3)
bm3("bar")

'bar'

StackOverFlow上[Adding a Method to an Existing Object Instance](https://stackoverflow.com/questions/972/adding-a-method-to-an-existing-object-instance/2982?r=SearchResults#2982)的第一个回答解释的非常好。

### 如何正确的继承字典、列表、字符串

平时要自定义字典、列表或者字符串的时候，一般我们会继承`dict`，`list`,`string`类，其实在`python`中，更推荐继承`collections`模块下的`UserDict`，`UserList`和`UserString`类，这几个类处理起来更容易，因为底层的字典、列表、字符串可以使用`data`属性来访问:

In [8]:
from collections import UserDict


class MyDict(UserDict):
    pass


d = MyDict(a=1, b=2)
d.data

{'a': 1, 'b': 2}

### 复制实例

实际编程中很少要复制一个实例，但是今天在重温《流畅的python》的时候，发现复制一个实例和直觉有一些冲突，特记录如下：

In [3]:
from copy import copy


class Bus:
    def __init__(self, passengers):
        self.passengers = [] if passengers is None else list(passengers)  # 对传入的列表进行复制

    def pick(self, passenger):
        self.passengers.append(passenger)

    def drop(self, passenger):
        self.passengers.remove(passenger)


bus1 = Bus(['Alice', 'John', 'Tom'])
bus2 = copy(bus1)
bus1.passengers.append('Alex')
bus1.passengers
bus2.passengers

['Alice', 'John', 'Tom', 'Alex']

['Alice', 'John', 'Tom', 'Alex']

直觉上，`passengers`的列表整个应该被复制，但显然不是这样，复制的仅仅只是列表的引用。可以这样记忆，当浅复制一个实例时，实例的属性就像是列表的元素，会对每个元素进行复制，如果属性引用的是一个可变对象，那么和列表的复制一样，复制的是这个可变对象的引用。

### 类属性所在的局部作用域

学习django的时候看到一个类属性的有趣用法：
```python
class Person(models.Model):
    SHIRT_SIZES = (
    ('S', 'Small'),
    ('M', 'Medium'),
    ('L', 'Large'),
    )
    name = models.CharField(max_length=60)
    shirt_size = models.CharField(max_length=1, choices=SHIRT_SIZES)
```
`shirt_size`直接引用了同样是类属性的`SHIRT_SIZES`，就好像是一个作用域一样。然后查资料，有这么一句：
> 尽管类能够访问外层函数的作用域，但它们不能作为类中其它代码的外层作用域：Python 搜索外层函数来访问被引用的变量，但从来不会搜索外层类。也就是说，类是一个可以访问其外层作用域的局部作用域，但其本身却不能作为一个外层作用域被访问。因为方法函数中对变量的搜索跳过了外层的类，所以类属性必须作为对象属性并使用继承来访问。

说明整个类实际上确实是一个局部作用域，类属性之间是可以互相访问的，而且可以访问到外层作用域的。比如：
```python
a = 42

class C:
    b = a
    
C.b
```
输出为42。可见，类属性所在的区域是可以访问到外层作用域的，这块区域就像是一个局部作用域，但是它不能作为类中其它代码的外层作用域，比如：
```python
class C:
    a = 42
    
    def __init__(self):
        print(a)
```
这里会抛出`NameError`，因为`__init__`方法会直接跳过`C`类的局部作用域到外层作用域去查找`a`。

### 手动给实例绑定方法

函数只能是类的属性，通过实例调用的时候才会自动的将实例作为第一个参数传递给函数，因此可以像下面这样手动的给类绑定方法：

In [8]:
def add(self):
    return self.a + self.b


class C:
    def __init__(self, a, b):
        self.a = a
        self.b = b


C.add = add
c = C(2, 3)
c.add()

5

但是不能直接给实例添加函数属性，实例的函数属性就是一个普通的函数，不会将实例作为第一个参数传递给函数：

In [9]:
def add(self):
    return self.a + self.b


class C:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
        
c = C(2, 3)
c.add = add
c.add()

TypeError: add() missing 1 required positional argument: 'self'

如果要给实例绑定一个函数，可以用`types.MethodType`进行绑定：

In [31]:
import types


def add(self):
    return self.a + self.b


class C:
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
c = C(2, 3)
c.add = types.MethodType(add, c)
c.add()

5

### 让对象支持上下文协议

《python cookbook》8.3节有详细介绍，这里补充一点就是，如果在`__exit__`中返回`True`，则错误不会传播，否则错误仍然会向上传播：

In [6]:
class C:
    def __init__(self, name):
        self.name = name
        
    def __enter__(self):
        print("in __enter__")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("in __exit__")
        return True
    
    
with C("telecomshy") as c:
    print(c.name)

in __enter__
telecomshy
in __exit__


有个细节要注意，实际上`with`是在实例上调用`__enter__`函数，所以上面的例子，完全可以写成这样：

In [7]:
c = C("shy")

with c:
    print(c.name)

in __enter__
shy
in __exit__


### `isinstance`判断是否为一组类的实例

`isinstance`不仅可以判断是否是某一个类的实例，第二个参数还可以传入一个元组，判断是否是一组类的实例：

In [6]:
isinstance([1, 2, 3], (list, tuple))

True

### 为什么书上传入的可变参数在内部总是先转换成列表

在很多书上看到一些类的代码，如果传入的参数是可变对象，总是在内部使用`list`转换一下，主要原因是因为可变对象作为参数的话，传递的是对象的引用：

In [1]:
arr = [1, 2, 3]

class C:
    def __init__(self, arr):
        arr.append(4)
        
C(arr)
arr

[1, 2, 3, 4]

可见，由于传递的是可变对象的引用，如果在内部改变了可变对象，则外部的可变对象也跟着改变了，一般而言，这是我们不希望见到的。因此，在内部通过`list`创造一个副本：

In [5]:
arr = [1, 2, 3]

class C:
    def __init__(self, arr):
        arr = list(arr)  # 创建一个副本
        arr.append(4)
        print(arr)
        
C(arr)
print(arr)

[1, 2, 3, 4]
[1, 2, 3]


## 模块和包

### `.`在脚本中到底代表什么

首先要明确一点，在一个脚本中，`.`永远代表的是用户当前所在的目录，而不是执行脚本文件所在的目录。这一点和脚本是不是包内文件，以及是否直接直接脚本还是以`-m`模式执行脚本没有一点关系。因此，这就会导致一个问题，比如我们在脚本中读取一个数据文件，使用相对目录，但是当我们不在该目录执行脚本时，就会抛出错误。比如下面的文件结构：

在`moduleA.py`中，我们读取data.txt，`open('data.txt', 'rt')`，此时如果我们在pkg中执行这个脚本，是没有问题的。但是当我们在pkg外执行这个脚本，比如`python pkg\moduleA.py`，会提示找不到文件，因为此时，`.`相当于在pkg外。

再次强调，`.`和包、相对导入绝对导入没有关系，它总是表示用户当前所在目录。

然后我们再来看导入模块，python解释器是如果定位目录的。当我们以普通的`python`方式执行脚本，则总是把脚本所在的目录加入`sys.path`，而如果我们以`python -m`的方式执行脚本，则和`.`一样，总是把用户的当前目录加入`sys.path`。因此，如果在moduleA.py中，使用`import moduleB`导入moduleB模块，不论用户在哪里，使用`python pkg/moduleA.py`还是`python moduleA.py`都不会报错，因为总是把moduleA.py文件所在目录加入`sys.path`，但是如果使用`python -m`的方式，则不一定。如果在pkg外执行，则会抛出错误，因为此时是把用户所在目录加入`sys.path`。

因此，如果我们在一个文件夹内，注意：文件夹，而不是包，导入同目录下的文件，要么使用相对导入，如果使用绝对导入，则需要从包的根目录开始导入，比如上面的结构，在moduleA.py中要导入moduleB.py中的模块，要么`from .moduleB import`，要么`from pkg.moduleB import`。

比如，在moduleA.py中，我们如果`from moduleB import func`，在moduleC.py文件中，又`from pkg import moduleA`。如果我们执行`python moduleC.py`，此时会报错，因为解释器此时只会把moduleC.py的目录加入到`sys.path`，此时`from pkg import moduleA`可以成功，但是`from moduleB import func`就会失败了。因为pkg并不在`sys.path`中。

注意，pkg文件夹中并没有`__init__.py`文件，实际上，此时pkg为称为一个命名空间包。只要理解python导入的规则，具体是不是一个包就无所谓了。

以上看起来很复杂，其实只要捋清楚，记住一点就行了：使用`python script.py`这种方式执行脚本时，会把`script.py`脚本所在目录添加到`sys.path`，会在脚本所在目录查找模块。其它任何情况，不管是导入模块，还是读取文件，所谓的相对目录都是相对于用户当前所在的目录。

### 包的相对导入

相对导入是针对包内部模块的互相导入的语法，包里的文件调用包里其它文件的时候，主文件（就是直接执行的脚本）不能放在包里面，包里只能放用来调用（即`import`）的文件。如以下的文件结构：

假设`c.py`里面有一个函数`func`,`b.py`文件通过相对语句导入`c.py`里面的函数`from ..c import func`，此时如果想要执行`b`文件（即将`b`当作直接执行的脚本），不能在`b.py`的相同文件夹下直接运行`b.py`。只能在包外，`pkg`的上级目录下，以模块方式执行`b.py`文件。如:`python -m pkg.mod.b`，`-m`是模块导入，所以此时`pkg`也要是一个包。

注意：以模块方式执行的时候，`b.py`文件的`__name__`依然是`__main__`，并没有变成了`b`（模块本身的名称）。

在python3.0中，包里面的文件如果执行绝对导入，按道理会跳过包本身。如在`c.py`文件中：`import a`，此时只会在`sys.path`的路径下查找`a`模块，不会在包内（即和`c.py`相同目录下）查找`a`模块，但是实际上发现是导入包内的模块。

### `python -m`和直接`python`的区别

`python -m`是以包模式运行文件，两者最大的区别是python解释器查找这个文件的路径不同，直接运行`python xxx.py`是把`xxx.py`文件所在的目录加入`sys.path`，而`python -m xxx.py`把当前工作目录加入`sys.path`。比如有如下文件结构（注意，如果要使用`python -m dir1.xxx`，则`dir1`文件夹下必须要有`__init__.py`文件）： 

假设此时用户在`dir0`目录下，如果在`xxx.py`文件中有`import yyy`语句，如果直接运行`python dir1\xxx.py`，会报错，找不到`yyy`模块，因为`xxx.py`的所在的`dir1`目录下没有`yyy.py`文件，但是`python -m dir1.xxx`，可以导入`yyy`模块，因为用户的工作目录是`dir0`，`dir0`下有`yyy`文件。  
不管哪种导入，此时`xxx.py`的模块名都是`__main__`，所以文件内的`if __name__ == "__main__"`语句依然为真。

另外，如果脚本中有相对导入的`import`语句，意味着这个脚本是一个包内的文件。如果想直接运行这个脚本，也只能使用包模式，即`python -m xxx.py`来执行这个文件。

### 如何导入上一级目录的模块

我们把上个问题再深入一点，仍然以上一个问题的文件结构为例，然后假设`dir0`在`D`盘下，文件结构如下：

先假设此时用户在`dir1`文件夹下，`xxx.py`文件如下：

```python
import sys
import yyy

print(sys.path)
```

运行脚本，`python xxx.py`，此时显然会报错，我们把`import yyy`注释掉，看看此时的`sys.path`有什么，可见此时`sys.path`包含`'D:\\dir0\\dir1'`，这个目录下显然没有`yyy`文件，所以会报错。我们再改一改，改成这样： 
```python
import sys
sys.path.append("..")
import yyy

print(sys.path)
```
现在运行正常了，且`'D:\dir0\dir1'`和`..`都在`sys.path`列表中。别着急，还没完，现在上一层目录，退到`dir0`文件夹下，再运行`python dir1/xxx.py`，出乎意料，又报模块没有找到的错误，此时把`sys.path.append("..")`改成下面这句：
```python
sys.path.append((os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))))
```
此时运行成功，观察`sys.path`，发现此时`D:\\dir0`和`D:\dir1`都在`sys.path`列表中。主要原因如下，有点烧脑：  
直接使用`python xxx.py`方式运行脚本，首先会自动把`xxx.py`这个文件所在的绝对目录加入到`sys.path`中，不会管此时用户在哪个文件夹，但为什么使用`sys.path.append("..")`把`".."`加入`sys.path`，仍然会报错呢，`".."`不是代表文件的上级目录，也就是`yyy.py`文件所处的位置吗？
不对，此时`".."`不是指`"xxx.py"`文件所在目录的上级目录，而是指用户目前所在目录的上层目录。这个例子中，当你在`dir0`，`..`代表`dir0`的上级目录，即`D:`盘下，此时肯定找不到`yyy.py`，所以会报错。此时只有用`sys.path.append((os.path.abspath(os.path.join(os.path.dirname(__file__), '..'))))`，先定位到`xxx.py`文件所在目录，然后将文件的上级目录加入`sys.path`才行。

### 根据字符串和文件路径导入模块

如果只是根据字符串来导入模块，使用`importlib`的`import_module`方法就可以：
```python
importlib.import_module("os")
```
注意，此时模块的路径还是遵循`python`的查找规则，只不过`import`语句编程字符串而已。  

如果想要根据文件的路径来导入模块，可以像下面这样做：
```python
from importlib import util
spec = util.spec_from_file_location("stub", "/ars/python/stub.py")
module = util.module_from_spec(spec)
spec.loader.exec_module(module)
```
注意`spec_from_file_location`的参数，第一个参数为模块名，第二个参数为模块的绝对路径。

另外，值得注意的是，`importlib.utils`还有一个`find_spec(module_name)`方法，可以用来判断模块是否存在，如果存在的话，返回`spec`（`spec`是什么没有完全弄明白，只知道它包含了模块导入的很多信息），如果不存在的话，则返回`None`。

### 模块导入的循环依赖

- [Python Circular Imports](https://stackabuse.com/python-circular-imports/)
- [Importing Python Modules](http://effbot.org/zone/import-confusion.htm)

上面的文章讲的已经比较清楚，下面通过代码加深理解，首先python导入一个模块的流程如下：
1. 首先我们运行程序的时候，解释器会先在内存里创建一个`sys.modules`的字典，键是模块名称，值是模块对象本身，如下：

```python
import sys
from pprint import pprint

pprint(sys.modules)
```
输出为：
```
{'__main__': <module '__main__' from 'main.py'>,
 '_abc': <module '_abc' (built-in)>,
 ...  # 还有大量模块省略
 'abc': <module 'abc' from 'D:\\programs\\Anaconda3\\lib\\abc.py'>,
 'builtins': <module 'builtins' (built-in)>,
 ...
}
```
2. 当我们运行一个脚本的时候，不管这个脚本的名字是什么，会在`sys.modules`增加一条记录，键是`__main__`，值就是这个脚本模块。
3. 当我们在脚本里导入另一个模块，会根据模块的名称查找`sys.modules`字典，看是否存在，如果存在，直接引用已存在的模块对象。
4. 如果不存在，则先创建一个空对象（本质是一个字典，键是模块名，值是这个模块对象），然后把这个空对象插入到`sys.modules`字典。
5. 加载模块的代码对象。
6. 在新的模块命名空间中执行模块代码对象，代码产生的所有变量都可以通过模块对象引用。

现在假设同一文件夹内有3个文件，分别是x.py，y.py，main.py，内容分别如下：
```python
# x.py的内容
import y

def xfunc():
    print("I am xfunc")
    
y.yfunc()

# y.py的内容
import x

def yfunc():
    print("I am xfunc")
    
x.xfunc()

# main.py的内容
import x
```
首先执行x.py，此时出现的提示是`y`找不到`yfunc`函数：
```
Traceback (most recent call last):
  File "x.py", line 1, in <module>
    import y
  File "D:\y.py", line 1, in <module>
    import x
  File "D:\x.py", line 6, in <module>
    y.yfunc()
AttributeError: module 'y' has no attribute 'yfunc'
```
现在我们执行main.py文件，发现此时的错误提示变成`x`找不到`xfunc`函数：
```
Traceback (most recent call last):
  File "main.py", line 1, in <module>
    import x
  File "D:\x.py", line 1, in <module>
    import y
  File "D:\y.py", line 6, in <module>
    x.xfunc()
AttributeError: module 'x' has no attribute 'xfunc'
```
为什么错误提示不同，来分析一下，第一种情况，执行的是x.py文件：
1. x.py作为脚本执行，先创建了一个`{'__main__':{}}`的空对象并插入`sys.modules`，开始执行代码。
2. 第一行`import y`，则创建一个`{'y': {}}`空对象，插入`sys.modules`，开始执行y.py的代码。
3. y.py的第一行是`import x`，又创建了一个`{'x': {}}`空对象，插入`sys.modules`，开始执行x.py的代码。注意，x.py实际相当于在`sys.modules`中创建了2个空对象，一个是`{'__main__':{}}`，一个是`{'x': {}}`，但是模块对象的代码还没有执行完，都还没有生成。
4. 此时x.py第一行`import y`，会直接从`sys.modules`获取`{'y': {}}`空对象，因为此时y.py根本没有执行完，模块的各种变量都还没有生成。
5. x.py继续往下执行，运行到`y.yfunc()`时，由于y模块的各种变量还没有生成，因此会报错：`module 'y' has no attribute 'yfunc'`。

第二种情况，执行的是main.py文件：
1. main.py作为脚本执行，创建一个`{'__main__':{}}`的空对象并插入`sys.modules`，开始执行代码。
2. 第一行`import x`，创建`{'x': {}}`空对象，插入`sys.modules`，开始执行x.py的代码。
3. x.py第一行`import y`，创建一个`{'y': {}}`空对象，插入`sys.modules`，开始执行y.py的代码。
4. y.py第一行`import x`，此时`sys.modules`已经有`x`模块，因此直接获取这个模块（但此时模块是空的，内部变量都还没有生成）的引用，继续往下执行。
5. 执行到`x.xfunc()`，由于引用的x模块是空的，所以抛出错误，找不到`xfunc`函数。

解决循环依赖最简单的方法就是把涉及到所有循环依赖的`import`导入语句放在文件最后，这样当导入发生的时候，所有的变量已经生成。

注意：导入语句并不是在当前模块的命名空间生成一个变量，比如在main.py中`import x`，并不是在`main`这个命名空间生成一个变量`x`，而是在`sys.modules`字典中添加一条记录。简单来说，`import`语句是将模块注册到`sys.modules`字典中去。另外，还有更好的解决方案，具体可以看《编写高质量python的59条建议》一书中底52条《用适当的方法打破循环依赖关系》。

## 描述符

### 什么时候用描述符？和特性使用场景的差异

当很多属性有相同的逻辑的时候，此时用描述符比用特性`@property`好--因为此时用特性的话，每个特性都要编写相同的逻辑。比如这样的一个场景，定义一个学生类，各科成绩是其特性，成绩必须要在0-100分之间，如果用特性来写的话，只能这样：

In [9]:
class Student:
    @property
    def math(self):
        return self.mscore
    
    @math.setter
    def math(self, score):
        if not (0 <= score <= 100):
            raise ValueError("分数不能大于100小于0!")
        self.mscore = score
        
    @property
    def english(self):
        return self.escore
    
    @english.setter
    def english(self, score):
        if not (0 <= score <= 100):
            raise ValueError("分数不能大于100小于0!")
        self.mscore = score        

可见，特性有相同的验证逻辑，因此每个特性里面都要写相同的验证代码，此时可以用描述符来代替：

In [26]:
class Score:
    def __get__(self, inst, cls):
        if inst is None:
            return self
        return self.score
    
    def __set__(self, inst, value):
        if not (0 <= value <= 100):
            raise ValueError("分数不能大于100小于0!")
        self.score = value


class Student:
    math = Score()
    english = Score()

In [11]:
s1 = Student()
s1.math = 75
s1.math

75

使用描述符的话，就不用重复写相同的代码。但是可惜，上面的代码仍然有问题，如下，`s2`实例的`math`设置为89，此时`s1`实例的`math`属性也成了89：

In [12]:
s2 = Student()
s2.math = 89
s2.math
s1.math

89

89

这是因为描述符是类属性，在类定义的时候就会执行`math = Score()`代码，所以`Student`的`math`属性其实都引用的是同一个描述符实例，可以这样解决：

In [47]:
class Score:
    def __init__(self):
        self.score = {}
    
    def __get__(self, inst, cls):
        if inst is None:
            return self
        return self.score[inst]
    
    def __set__(self, inst, value):
        if not (0 <= value <= 100):
            raise ValueError("分数不能大于100小于0!")
        self.score[inst] = value


class Student:
    math = Score()
    english = Score()

In [48]:
s1 = Student()
s1.math = 75
s1.math
Student.math.score

75

{<__main__.Student at 0x17e26aa5e08>: 75}

上面的解决方案仍然有最后一个小问题，就是会引起内存泄漏，因为描述符的`score`字典引用了`s1`,`s2`这样的实例，导致`s1`,`s2`实例不会被垃圾回收（反复运行上面一行的代码，可以发现每一次都新建了一个`Student`的实例，`Student.math.score`字典里的实例越来越多，无法回收）。因此最终的解决办法是将字典改为弱引用的：
```python
score = WeakKeyDictionary()
```

还有一种方法，即实际的属性不保存在描述符的实例里，而是保存在`Student`的实例中，如下：

In [67]:
class Score:
    def __init__(self, attrname):
        self.attrname = "_" + attrname
    
    def __get__(self, inst, cls):
        if inst is None:
            return self
        return getattr(inst, self.attrname)
    
    def __set__(self, inst, value):
        if not (0 <= value <= 100):
            raise ValueError("分数不能大于100小于0!")
        setattr(inst, self.attrname, value)


class Student:
    math = Score("math")
    english = Score("english")

上面这种应该是最简单的了，但是还有个小遗憾就是每次定义描述符，都要传入一个和属性名相同的字符串，显得有点冗余，可以用元类解决：

In [4]:
class Meta(type):
    def __new__(meta, clsname, bases, clsdict):
        for key, value in clsdict.items():
            if isinstance(value, Score):
                value.attrname = "_" + key
        return type.__new__(meta, clsname, bases, clsdict)


class Score:
    def __get__(self, inst, cls):
        if inst is None:
            return self
        return getattr(inst, self.attrname, 0)

    def __set__(self, inst, value):
        if not (0 <= value <= 100):
            raise ValueError("分数不能大于100小于0!")
        setattr(inst, self.attrname, value)


class Student(metaclass=Meta):
    math = Score()
    english = Score()

In [5]:
s1 = Student()
s2 = Student()
s1.math = 75
s1.math
s2.math = 89
s2.math
Student.math.__dict__

75

89

{'attrname': '_math'}

### 描述符的几个主要应用场景

#### 使用仅有`__set__`方法的描述符进行验证

当要对类的属性进行验证，此时可以使用只包含`__set__`方法的描述符，`__set__ `方法应该检查`value`参数获得的值，如果有效，使用描述符实例的名称为键， 直接在实例的`__dict__ `属性中设置。读取的时候则不需要经过`__get__`的处理：

In [81]:
class D:
    def __set__(self, inst, value):
        if isinstance(value, str):
            print("I am StrData descriptor")
            inst.__dict__["data"] = value
        else:
            raise TypeError("data属性要是字符串")


class C:
    data = D()


c = C()
c.data = "foo"
c.data
try:
    c.data = 123
except Exception as e:
    print(e)

I am StrData descriptor


'foo'

data属性要是字符串


注意，设置实例属性一定要用`inst.__dict__["data"] = value`的形式，不能直接`inst.data = value`，因为`inst.data`又会触发`__set__`方法，从而陷入无限循环。

#### 使用仅有`__get__`方法的描述符进行数据的高速缓存

当某一个属性需要进行复杂的计算，可以使用仅有`__get__`方法的描述符，在第一次读取属性的时候进行计算，然后将结果保存为实例的与描述符同名的属性，后期就会直接读取实例属性，不会再触发`__get__`方法，从而实现了数据的高速缓存：

In [13]:
class D:
    def __get__(self, inst, owner):
        if inst is None:
            return self  # 如果从类访问描述符，则返回描述符本身
        else:
            print("computing....")
            inst.data = 42  # 42代表计算出的结果
            return 42


class C:
    data = D()


c = C()
c.data  # 第一次读取data属性，此时c.data是描述符，触发__get__方法，进行计算，返回结果，并保存为实例的同名属性
c.data  # 第二次读取data属性，此时c.data是实例属性，不会再触发__get__方法，从而实现了数据的高速缓存

computing....


42

42

上述的场景都是利用了描述符`__set__`和`__get__`方法的不对称性，`__set__`方法是覆盖型方法，它会覆盖实例的同名属性，意思是写入属性的时候，即使实例有和描述符相同名字的属性，它也会进行拦截。而`__get__`方法是非覆盖的，如果实例有和描述符相同名称的属性，则读取的时候，不会再触发`__get__`方法，直接读取实例的属性。

### 覆盖型描述符和非覆盖描述符

我很长一段时间都搞错了一件事情，就是一直以为覆盖不覆盖指的是`__set__`和`__get__`方法，即`__set__`方法是覆盖型的，所有的写入操作都会拦截，而`__get__`方法是非覆盖型的，如果实例有和描述符一样的同名属性，则会优先读取实例的属性，而不会触发描述符的`__get__`方法。

最近突然发现自己的理解完全错了，其实是根据描述符有还是没有`__set__`方法，来决定是覆盖还是不覆盖，即，如果实例有同名属性，但是只要描述符有`__set__`方法，那么这个描述符是覆盖型的，不管是读还是写，都会进行拦截。而如果没有`__set__`方法，那么就是非覆盖型的描述符，如果有同名属性，会优先读取实例的属性。

In [3]:
class D:
    def __set__(self, inst, value):
        print("setting")
        inst.__dict__["data"] = value

    def __get__(self, inst, cls):
        print("getting")
        if inst is None:
            return self
        return inst.__dict__["data"]


class C:
    data = D()


c = C()
c.data = 42
print(c.__dict__)  # 实例已经有了data属性，但是因为描述符有`__set__`方法，此时读取的时候仍然会进行拦截，触发__get__方法
print(c.data)

setting
{'data': 42}
getting
42


In [4]:
class D:
    def __get__(self, inst, cls):
        print("getting")
        if inst is None:
            return self
        return inst.__dict__["data"]


class C:
    data = D()


c = C()
c.data = 42
print(c.__dict__)  # 实例已经有了data属性，因为描述符没有`__set__`方法，此时读取不会触发__get__方法。
print(c.data)

{'data': 42}
42


### 使用描述符做方法的装饰器

我们可以用函数，和类作装饰器，但是，在装饰方法的时候，类作为装饰器就不好用了，如下：

In [82]:
class Wrapper:
    def __init__(self, func):
        self.func = func

    def __call__(self, *args, **kwargs):
        print("in wrapper")
        return self.func(*args, **kwargs)


@Wrapper
def func(a, b):
    return a + b


func(2, 3)

in wrapper


5

使用`Wrapper`类包装方法时，会报错，因为此时`c.func`返回的是`Wrapper`的实例，然后再调用`Wrapper`实例的`__call__`方法，这个过程丢失了`C`的`self`实例的传递：

In [86]:
class C:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    @Wrapper
    def func(self):
        return self.a + self.b


c = C(2, 3)
try:
    c.func()
except TypeError as e:
    print(e)

in wrapper
func() missing 1 required positional argument: 'self'


此时给`C`类添加`__get__`方法，将其变为一个描述符：

In [93]:
class Wrapper:
    def __init__(self, func):
        self.func = func
        
    def __get__(self, inst, cls):
        def wrapper(*args, **kwargs):           
            print("in wrapper")
            return self.func(inst, *args, **kwargs)
        return wrapper


class C:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    @Wrapper
    def func(self):
        return self.a + self.b

c = C(2, 3)
c.func()

in wrapper


5

也可以使用一个包含`__get__`和`__call__`方法的类作为装饰器：

In [94]:
class Wrapper:
    def __init__(self, func):
        self.func = func
        
    def __call__(self, *args, **kwargs):
        print("in wrapper")
        return self.func(self.inst, *args, **kwargs)
        
    def __get__(self, inst, cls):
        self.inst = inst
        return self


class C:
    def __init__(self, a, b):
        self.a = a
        self.b = b

    @Wrapper
    def func(self):
        return self.a + self.b

c = C(2, 3)
c.func()

in wrapper


5

2022年1月30日补充：  
《python cookbook》有专门一节，使用类作装饰器，和上面的例子类似，但是有几个知识点值得一提：

1. 当方法或者函数是实例属性时，调用时不会绑定任何实例，所以上面的例子中，在`__call__`里面调用`self.func(*args)`，`Descriptor`的实例，不会作为第一个参数传递给`self.func`，这里`self.func`实际上是被装饰的`C`的`func`方法，所以当`self.func`调用时，传递的第一个参数应该是`c`的实例：

In [123]:
class C:
    def __init__(self):
        self.func = func
        
def func(*args):
    print(f"func invoked, args is {args}")
    
c = C()
c.func()  # args为空，func是实例属性，c不会作为第一个参数传递给func

func invoked, args is ()


2. 方法其实也是描述符。当使用`c.func()`这样的方式调用时，首先`c.func`会调用`func`的`__get__`方法，返回一个绑定了实例的`bound method`对象，当调用`bound method()`时，会把绑定的实例`c`作为第一个参数传递给`func`：

In [124]:
def func(self):
    print(f"self is {self}")
    
class C:
    pass

In [125]:
c = C()
bm = func.__get__(c, C)  
bm

<bound method func of <__main__.C object at 0x000001D023E03130>>

In [126]:
bm()

self is <__main__.C object at 0x000001D023E03130>


3. `MethodType`返回一个`bound method`，将实例绑定给一个函数，调用这个函数时，实例会作为第一个参数传递给函数，需要理解的事，可以多重嵌套：

In [127]:
def func(*args):
    print(args)

In [128]:
class C:
    pass

In [129]:
class D:
    pass

In [130]:
bm1 = types.MethodType(func, C())
bm2 = types.MethodType(bm1, D())

In [131]:
bm1

<bound method func of <__main__.C object at 0x000001D023E03FD0>>

In [132]:
bm2

<bound method func of <__main__.D object at 0x000001D023E03310>>

In [121]:
bm2()

(<__main__.C object at 0x000001D023E039A0>, <__main__.D object at 0x000001D023DE9970>)


所以，当使用`MethodType`绑定一个实例方法时，如下面的例子，`c.func`返回绑定了`c`的`bound method`，所以当最后调用时，`func`依次接收`c`和`d`两个实例：

In [70]:
class C:
    def func(self, *args):
        print(f"func invoked! self is {self}, args are {args}")
        
class D:
    pass

In [71]:
d = D()

In [72]:
types.MethodType(c.func, d)()

func invoked! self is <__main__.C object at 0x000001D023BBA670>, args are (<__main__.D object at 0x000001D023BBAAF0>,)


In [136]:
c.func

__get__ invoked!


<bound method ? of <__main__.C object at 0x000001D023DF37F0>>

最后我们看书上代码，这里增添了一些打印的信息，取消了`wraps`的使用方便理解，逐个注释以便理解：

In [133]:
import types

class Descriptor:
    def __init__(self, func):
        self.func = func  # 1. func相当于C的func方法，当self.func()这样调用时，self不会作为第一个参数传递给func
        
    def __call__(self, *args):
        # 4. bm()调用时，实际会调用__call__方法，第一个参数总是自身的实例，绑定的c是args的第一个参数
        print(f"__call__ invoked, self is {self}, args is {args}")
        # 5. 因此直接将args传递给self.func就好
        self.func(*args)
        
    def __get__(self, inst, cls):
        # 2 .装饰func以后，func是一个Descriptor的实例，当c.func()这样调用时，首先c.func会调用这个__get__方法
        print("__get__ invoked!")
        if inst is None:
            return self
        # 3. 这里将Descriptor实例本身和c实例绑定，返回一个bound methed，当bm()调用时，c会当作第一个参数传递给bm
        return types.MethodType(self, inst)  

In [134]:
class C:
    @Descriptor
    def func(self):
        print("func invoked!")

In [137]:
c = C()
# func是描述符，会调用它的__get__方法，返回的是绑定了c实例的一个bound method对象
c.func  

__get__ invoked!


<bound method ? of <__main__.C object at 0x000001D023DF3520>>

In [138]:
# 调用bound method对象会调用其包装的原始方法，就是调用描述符实例的__call__方法，并将绑定的实例c传递进去
c.func()

__get__ invoked!
__call__ invoked, self is <__main__.Descriptor object at 0x000001D023DF3610>, args is (<__main__.C object at 0x000001D023DF3520>,)
func invoked!


## 元类

### `__call__`，`__new__`，`__init__`的执行顺序

- [Using the __call__ method of a metaclass instead of __new__?](https://stackoverflow.com/questions/6966772/using-the-call-method-of-a-metaclass-instead-of-new)

首先思考一下平时在什么情况下，我们会调用`__call__`方法：

In [23]:
class C:
    def __call__(self):
        print("__call__")
        
c = C()
c()

__call__


可见，当运行c()调用实例时，会运行类的`__call__`方法，类其实就是元类的实例，因此当我们运行C()，同样的道理，必然也会调用C的类，也就是元类的`__call__`方法。推广一下，`__call__`,`__new__`,`__init__`的执行顺序是：先执行元类的元类的`__call__`方法，该方法依次调用元类的`__new__`方法和`__init__`方法，最后返回一个元类：

In [7]:
class MMCls(type):
    def __call__(metacls, clsname, base, attrs):
        print("enter mmclass __call__")
        cls = super().__call__(clsname, base, attrs)
        print("exit mmclass __call__")
        return cls


class MCls(type, metaclass=MMCls):
    def __new__(metacls, clsname, base, attrs):
        print("enter mclass __new__")
        cls = super().__new__(metacls, clsname, base, attrs)
        print("exit mclass __new__")
        return cls

    def __init__(cls, clsname, base, attrs):
        print("enter mclass __init__")
        super().__init__(clsname)
        print("exit mclass __init__")


class Cls(metaclass=MCls):
    pass

enter mmclass __call__
enter mclass __new__
exit mclass __new__
enter mclass __init__
exit mclass __init__
exit mmclass __call__


类创建实例的过程也是一样的，先调用元类的__call__方法，在元类的__call__方法中，依次调用类的__new__和__init__方法，最后返回一个实例。比如，当实现单例模式的时候，就可以通过元类的__call__方法来实现，这样的好处是，不会每次创建实例都会调用__init__方法，如下：

In [10]:
class C:
    _inst = None
    
    def __new__(cls, *args, **kwargs):
        if C._inst == None:
            inst = super().__new__(cls, *args, **kwargs)
            C._inst = inst
            return inst
        else:
            return C._inst
        
    def __init__(self):
        print("create a C instance")
        
c1 = C()
c2 = C()
c1 is c2

create a C instance
create a C instance


True

可见，虽然两次创建的是同一个实例，但是每次调用C()，都会执行一次init函数。

In [11]:
class MetaC(type): 
    def __init__(cls, *args, **kwargs):
        cls._inst = None
        super().__init__(*args, **kwargs)
        
    def __call__(cls, *args, **kwargs):
        if cls._inst == None:
            inst = super().__call__(*args, **kwargs)
            cls._inst = inst
            return inst
        else:
            return cls._inst
        
class C(metaclass=MetaC):
    def __init__(self):
        print("create a C instance")
        
c1 = C()
c2 = C()
c1 is c2

create a C instance


True

可见，只要创建的是同一个实例，则只会执行一次init函数。具体的例子见《Python Cookbook》9.13。

### 元类中`__new__`和`__init__`方法的区别

简单来说，`__new__`在创建类之前运行，`__init__`在创建类之后运行，`__init__`可以做的，`__new__`都可以做，反之则不然。对于`__new__`来说，四个参数：`metacls, clsname, supercls, attrsdict`缺一不可，但是对`__init__`来说，只需要`cls, clsname`两个参数就够了。 接下来通过两个主要的场景来理解它们之间的区别。
- 场景一：通过元类编程的方式让一个类继承另一个类  
此时这种情况就只能使用`__new__`,因为使用`__init__`的时候，类已经创建了，无法再修改它的父类。

In [26]:
class MetaCls(type):
    def __new__(meta, name, bases, attrs):  
        bases = bases + (C, )
        return type.__new__(meta, name, bases, attrs)
    
    def __init__(cls, name, bases, attrs):
        type.__init__(cls, name)


class C:
    a = "I am C's attribute!"

class D(metaclass=MetaCls):
    pass

In [29]:
D.__mro__

(__main__.D, __main__.C, object)

- 场景二：给类添加新的属性  
这种场景两个方法都可以，但是写法不同：

In [31]:
class MetaCls(type):
    def __new__(meta, name, bases, attrs):  
        attrs['a'] = 'I am in metacls __new__'
        return type.__new__(meta, name, bases, attrs)

class C(metaclass=MetaCls):
    pass

In [32]:
C.a

'I am in metacls'

在`__new__`中，由于类还没有创建，所以只能通过修改`attrs`的方式给类添加属性。

In [33]:
class MetaCls(type):
    def __init__(cls, name, bases, attrs):  
        # attrs['a'] = 'I am in metacls' 不能通过此种方式，此时类已经创建，attrs只是保存类属性的一个字典，修改它并不会改变类的属性
        cls.a = 'I a in metacls __init__'
        return type.__init__(cls, name)

class C(metaclass=MetaCls):
    pass

In [34]:
C.a

'I a in metacls __init__'

注意，在`__init__`中，类已经创建，不能通过直接修改`attrs`字典的方式来改变类的属性（你会发现根本不起作用），只能通过`cls.attr`的方式来修改。

### 一些元类的例子

#### 动态将函数添加为类的静态属性

In [36]:
def a():
    return 'a'

def b():
    return 'b'

In [37]:
class Meta(type):
    def __new__(metacls, clsname, bases, attrs):
        for func in [a, b]:
            attrs[func.__name__] = staticmethod(func)
        return type.__new__(metacls, clsname, bases, attrs)

class C(metaclass=Meta):
    pass

动态将函数添加为类的普通属性：

## 异常和`traceback`对象

### `raise`，`except`抛出和捕捉的是类还是实例

`raise`可以不加参数直接抛出异常，`except`捕获的对象看起来也是一个类，但实际上`raise`和`except`抛出和捕获的都是一个实例：

In [14]:
try:
    raise TypeError
except TypeError as e:
    print(isinstance(e, TypeError))

True


### 异常的显示

传递给任何异常（不管是内置异常还是自定义的异常）构造函数的参数都会保存在实例的`args`元组属性中，默认情况下，打印该实例的只是把实例的`args`属性打印出来，也就是说，`Exception`的`__str__`类似这样：
```python
def __init__(self, args):
    self.args = args

def __str__(self):
    return self.args
```
可以通过重写异常类的`__str__`或者`__repr__`方法来返回想要为异常显示的字符串：

In [50]:
try:
    raise ValueError('Oops, something bad happend!')
except Exception as e:
    print(e)
    print(e.args)

Oops, something bad happend!
('Oops, something bad happend!',)


In [4]:
class MyException(Exception):
    def __str__(self):
        return 'I really love you!'

try:
    raise MyException("I dont't love you!") # raise语句其实执行了MyException()，建立了一个异常类的实例
except Exception as e:
    print(e)
    print(e.args)

I really love you!
("I dont't love you!",)


### `StopIteration`异常

单独把`StopIteration`异常拿出来讲，主要是因为这个异常和其它异常相比，有一个不同的而且是相当重要的应用场景，当生成器，也就是协程结束的时候，会自动抛出`StopIteration`异常，此时，`StopIteration`会捕获协程最后结束时的返回值，把返回值作为参数（下面例子中的`args`），传递给`StopIteration`：

In [1]:
def c():
    yield
    return 42

g = c()
try:
    next(g)
    next(g)
except StopIteration as e:
    print(e.args)
    print(e.value)

(42,)
42


那么，`value`属性又是哪里来的呢？注意，`value`总是`args`属性的第一个值，其它的异常是没有`value`属性的：

In [2]:
s = StopIteration(1, 2, 3)
s.args
s.value

(1, 2, 3)

1

因此，生成器中的`return expr`表达式会触发`StopIteration(expr)`异常抛出，注意，将`expr`的值作为参数传递给`StopIteration`异常。

### `traceback`模块和`traceback`对象

某些情况下，单纯的打印异常所能提供的信息会非常有限，此时需要更详细的错误信息。可以使用`traceback`模块提供的接口来提取、格式化和打印`Python`程序的堆栈跟踪结果。  
一般情况下，当抛出异常时，我们可以通过`sys.last_traceback`或者`sys.exc_info()`的第三个返回值来获取`traceback`对象，然后通过`traceback`模块提供的各种接口来处理异常：

In [17]:
import traceback
from traceback import print_tb, print_exception, print_exc
import sys

In [24]:
import sys
try:
    raise TypeError("I am a TypeError")
except TypeError as e:
    exc_type, exc_value, tb = sys.exc_info()
    print(tb.tb_lineno)
    print(exc_type)
    print(exc_value)   
    print(tb)  # 直接打印tb只是显示<traceback object at 0x0000017E2491B948>，无法打印traceback对象的内容
    print_tb(tb)

3
<class 'TypeError'>
I am a TypeError


  File "<ipython-input-24-657d83866b4f>", line 3, in <module>
    raise TypeError("I am a TypeError")


可见，平时如果执行程序出错，抛出的信息里面是包含了异常类型，异常的值以及`traceback`三类信息的，如果直接`print(e)`的话，仅仅只是把异常的值打印出来。这里稍微提一句`sys.exc_info()`和`sys.last_traceback`的区别以及使用场景，`sys.exc_info()`主要在`except`跟着的代码块里使用，分别返回异常类型，异常值和`traceback`对象构成的元组。而`sys.last_traceback`仅用在交互式的场景下面，当运行的代码抛出一个错误时，可以接着使用`sys.last_traceback`获取最后一个`traceback`对象，注意，不能用在`except`后面的代码块中，另外还有`sys.last_type`和`sys.last_value`分别对应异常类型和异常值。  

`traceback`模块主要关注4个方法：
- `print_tb(tb, limit=None, file=None)`：打印`traceback`对象。
- `print_exception(etype, value, tb, limit=None, file=None, chain=True)`：打印指定的异常类型，异常值和`traceback`对象。
- `print_exc(limit=None, file=None, chain=True)`：`print_exception`的简写，直接打印`sys.exc_info()`返回的三元组。
- `traceback.format_exc(limit=None, chain=True)`：类似`print_exc()`，但是不是打印，而是返回一个字符串。

其中`limit`代表堆栈回溯的几层，`None`代表全部打印，`file`指定输出文件，也可以指定为`stdout`等类文件接口，默认输出到`stderr`。

In [4]:
from traceback import print_exc, print_tb, format_exc


def raise_typerror():
    raise TypeError("I am a TypeError")


try:
    raise_typerror()
except TypeError as e:
    print_exc(limit=1)

Traceback (most recent call last):
  File "<ipython-input-4-9698c52f62d7>", line 9, in <module>
    raise_typerror()
TypeError: I am a TypeError


参考文档[《Python Traceback详解》](https://www.jianshu.com/p/a8cb5375171a)。

In [38]:
from functools import partial

class Meta(type):
    def __new__(metacls, clsname, bases, attrs):
        for func in [a, b]:
            # 注意两点：
            # 1.因为要多传递一个self参数，所以通过回调函数传参
            # 2.回调函数必须使用func=func的方式将func赋值给本地，否则关联的永远是最后一个func
            def wrapper(func=func, *args, **kwargs): 
                return func(*args[1:], **kwargs)            
            attrs[func.__name__] = partial(wrapper, func)
        return type.__new__(metacls, clsname, bases, attrs)

class D(metaclass=Meta):
    pass